In [1]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 264 kB in 4s (62.9 kB/s)
Reading 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.5.1.jar

--2022-12-29 07:06:30--  https://jdbc.postgresql.org/download/postgresql-42.5.1.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1046770 (1022K) [application/java-archive]
Saving to: ‘postgresql-42.5.1.jar.2’

postgresql-42.5.1.j 100%[===================>]   1022K  5.98MB/s    in 0.2s    

2022-12-29 07:06:30 (5.98 MB/s) - ‘postgresql-42.5.1.jar.2’ saved [1046770/1046770]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.5.1.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Kitchen_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   37000337|R3DT59XH7HXR9K|B00303FI0G|     529320574|Arthur Court Pape...|         Kitchen|          5|            0|          0|   N|                Y|Beautiful. Looks ...|Beautiful.  Looks...| 2015-08-31|
|         US|   15272914|R1LFS11BNASSU8|B00JCZKZN6|     274237558|Olde Thompson Bav...|         Kitchen|          5|    

In [31]:
kitchen_df = df.limit(40000)

### Create DataFrames to match tables

In [32]:
from pyspark.sql.functions import to_date


In [33]:
# Create the customers_table DataFrame
from pyspark.sql.functions import sum,avg,max,count
customers_df = kitchen_df.groupby(df["customer_id"]).agg(count('customer_id')).withColumnRenamed('count(customer_id)',"customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   37000337|            15|
|   15272914|             1|
|   36137863|             1|
|   43311049|             5|
|   13763148|             1|
|   19009420|             1|
|   40599388|             1|
|   22719359|             5|
|   47478640|             3|
|   34195504|             4|
|   19100570|             1|
|   10299811|             1|
|   32687006|             2|
|   43260893|             1|
|    8067227|             6|
|   18139929|             1|
|   12282702|             1|
|   38649737|             1|
|   33460969|             1|
|   40274860|             1|
+-----------+--------------+
only showing top 20 rows



In [34]:
# Create the products_table DataFrame and drop duplicates. 
products_df = kitchen_df.select(['product_id', 'product_title']).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00303FI0G|Arthur Court Pape...|
|B00JCZKZN6|Olde Thompson Bav...|
|B00JLIKA5C|Progressive Inter...|
|B000GBNB8G|Zyliss Jumbo Garl...|
|B00VJ5KX9S|1 X Premier Pizza...|
|B00HYB5YY0|       CHEF Aluminum|
|B000HEBAV2|Presto Dual Profr...|
|B0012DS4GG|Rubbermaid Produc...|
|B00FLQ4EE6|Cuisinart 12-Piec...|
|B00DBS9OTG|Kegco 6" Stainles...|
|B00AN9UJ68|Cuisinart Smart S...|
|B00L2P0KNO|Searzall Torch At...|
|B00NQOJQXY|Wilton 1512-1664 ...|
|B00080QE1Q|Magnalite Classic...|
|B009VU17ZM|Sun's Tea(TM) 20o...|
|B00MY71KO2|Adventure Time Be...|
|B00A6N18CK|DecoBros 3 Tier D...|
|B00AB8NOLS|Brita Water Filte...|
|B00FB4UPA0|Nifty Home 24 K-C...|
|B00YMHBQ0A|Eoonfirst Snoopy ...|
+----------+--------------------+
only showing top 20 rows



In [35]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = kitchen_df.select(['review_id','customer_id', 'product_id', 'product_parent', to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3DT59XH7HXR9K|   37000337|B00303FI0G|     529320574| 2015-08-31|
|R1LFS11BNASSU8|   15272914|B00JCZKZN6|     274237558| 2015-08-31|
|R296RT05AG0AF6|   36137863|B00JLIKA5C|     544675303| 2015-08-31|
|R3V37XDZ7ZCI3L|   43311049|B000GBNB8G|     491599489| 2015-08-31|
|R14GU232NQFYX2|   13763148|B00VJ5KX9S|     353790155| 2015-08-31|
| RZQH4V7L2O1PL|   19009420|B00HYB5YY0|     432241873| 2015-08-31|
|R1F8JMOSPJ3KO7|   40599388|B000HEBAV2|     584680984| 2015-08-31|
|R1ZISGY2BWW4Z5|   22719359|B0012DS4GG|     772637306| 2015-08-31|
|R17PW4I3AE5WZW|   47478640|B00FLQ4EE6|     755416578| 2015-08-31|
|R3D93G1KTP6A8P|   34195504|B00DBS9OTG|     648762742| 2015-08-31|
|R18TQIW1NKPUNU|   19100570|B00AN9UJ68|     495720940| 2015-08-31|
|R34KUNL21WU248|   10299811|B00L2P0KNO|      41330497| 2015-08

In [36]:
# Create the vine_table. DataFrame
vine_df = kitchen_df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3DT59XH7HXR9K|          5|            0|          0|   N|                Y|
|R1LFS11BNASSU8|          5|            0|          1|   N|                Y|
|R296RT05AG0AF6|          5|            0|          0|   N|                Y|
|R3V37XDZ7ZCI3L|          5|            0|          1|   N|                Y|
|R14GU232NQFYX2|          5|            0|          0|   N|                Y|
| RZQH4V7L2O1PL|          1|            1|          1|   N|                Y|
|R1F8JMOSPJ3KO7|          5|            0|          0|   N|                Y|
|R1ZISGY2BWW4Z5|          5|            0|          0|   N|                Y|
|R17PW4I3AE5WZW|          5|            0|          0|   N|                Y|
|R3D93G1KTP6A8P|          3|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [37]:
# Configure settings for RDS
# Store environmental variable
from getpass import getpass
mode = "append"
password = getpass("Enter!")
jdbc_url="jdbc:postgresql://db-challenge.c5xoi7hhyfmj.us-west-2.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}


Enter!··········


In [40]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [41]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='product_table', mode=mode, properties=config)

In [42]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [43]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)